In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import sklearn
import time

In [2]:
train_fn = "/content/drive/MyDrive/bank_train.csv" #train path
test_fn = "/content/drive/MyDrive/bank_test.csv" #test path

In [3]:
train_df = pd.read_csv(train_fn)
test_df = pd.read_csv(test_fn)

Marketing Preprocessing (only run when using marketing dataset)

In [4]:
# Code categorical variables
for p in ["job", "marital", "education", "default", "housing", "loan", "contact", "poutcome", "month"]:
	label_encoder = sklearn.preprocessing.LabelEncoder()
	label_encoder.fit(train_df[p].unique())
	train_df[p] = label_encoder.transform(train_df[p])
	test_df[p] = label_encoder.transform(test_df[p])


loan Preprocessing (only run when using marketing dataset)

In [ ]:
# Code categorical variables
for p in ["grade", "sub_grade", "initial_list_status"]:
	label_encoder = sklearn.preprocessing.LabelEncoder()
	label_encoder.fit(train_df[p].unique())
	train_df[p] = label_encoder.transform(train_df[p])
	test_df[p] = label_encoder.transform(test_df[p])

In [5]:
cols = train_df.columns.values 

In [6]:
predictors = [p for p in cols if p != "y"]
target = "y"

In [7]:
# Parameters for random forest
# TODO make these optional command line arguments
n = 500
split_n = 85
leaf_n = 75
seed = 1995
jobs=2 #increase to desired parallel jobs number, or set to -1 to utilize maximum cores available

In [8]:

# Initalize random forest class
rf = RandomForestClassifier(random_state = seed, n_estimators = n, min_samples_split = split_n, min_samples_leaf = leaf_n, n_jobs=jobs)

In [9]:
# Train
%%time
rf.fit(train_df[predictors], train_df[target])

CPU times: user 16.8 s, sys: 13.8 ms, total: 16.8 s
Wall time: 16.9 s


RandomForestClassifier(min_samples_leaf=75, min_samples_split=85,
                       n_estimators=500, random_state=1995)

In [10]:
from time import sleep
from random import random
from multiprocessing.pool import ThreadPool

import threading 
import time 
import array
lock = threading.Lock()

counter=0
total=0

In [13]:
#test
def model_test_time(name):
    global counter, total, test_probs
    start = time.time()
    test_probs = rf.predict_proba(test_df[predictors])
    end = time.time()
    time_per_thread = end-start
    #lock.acquire
    counter+=1
    #lock.release
    print("Time spend by thread " + str(name) + ": " + str(time_per_thread))
    #lock.acquire
    total+=time_per_thread
    #lock.release
    print("Total time:" + str(total))

In [14]:
with ThreadPool() as pool:
    pool.map(model_test_time, range(10))

Time spend by thread 0: 1.0066688060760498
Total time:13.79395079612732
Time spend by thread 2: 1.0391135215759277
Total time:14.833064317703247
Time spend by thread 1: 1.1364021301269531
Total time:15.9694664478302
Time spend by thread 3: 1.1667635440826416
Total time:17.136229991912842
Time spend by thread 4: 0.9887890815734863
Total time:18.125019073486328
Time spend by thread 6: 0.9806187152862549
Total time:19.105637788772583
Time spend by thread 5: 1.0311942100524902
Total time:20.136831998825073
Time spend by thread 7: 1.0192201137542725
Total time:21.156052112579346
Time spend by thread 8: 0.7651762962341309
Total time:21.921228408813477
Time spend by thread 9: 0.7582409381866455
Total time:22.679469347000122


In [15]:
# Get predicted probability that y = 1
test_predictions = [x[1] for x in test_probs]

In [16]:
# Calculate MSE for test data
test_error = (test_predictions - test_df[target]) ** 2
mse = test_error.mean()

In [18]:
# Calculate misclassification rate on test data
mis_rate = 1 - accuracy_score(test_df[target], rf.predict(test_df[predictors]))

In [19]:
print (accuracy_score(test_df[target], rf.predict(test_df[predictors])))

0.8986109882332124


In [20]:
# Print out summary
print ("**************sklearn random forest**************")
print ("Test MSE: " + str(mse))
print ("Test misclassification rate: " + str(mis_rate))

**************sklearn random forest**************
Marketing data:
Test MSE: 0.06916433152426746
Test misclassification rate: 0.1013890117667876
